In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split

dataset_crop_weeds = '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/'

# Images Sizes
img_height = 512
img_width = 512

# Defines batch size and number of epochs
batch_size = 32
epochs = 10

# List to store image paths and annotations
images = []
annotations = []

for filename in os.listdir(dataset_crop_weeds):
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_crop_weeds, filename)
        images.append(image_path)
    elif filename.endswith('.txt'):
        annotation_path = os.path.join(dataset_crop_weeds, filename)
        annotations.append(annotation_path)   

# Converts annotations to DataFrame
# data = {'Annotation': annotations}
# df = pd.DataFrame(data)

2024-03-28 22:09:47.283612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 22:09:47.283779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 22:09:47.445397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Check the number of images and annotations loaded
print(f"Total Images: {len(images)}")
print(f"Total Annotations: {len(annotations)}")

Total Images: 1300
Total Annotations: 1300


In [5]:
# Split the data into training, validation, and test sets
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.1, random_state=42)

In [6]:
def load_and_preprocess_image(image_path):
    # Load image using OpenCV
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Resize image to desired dimensions (img_height, img_width)
    image = cv2.resize(image, (img_width, img_height))

    # Preprocess image further as needed (e.g., normalization)
    # Example: image = image / 255.0  # Normalize pixel values to [0, 1]

    return image

In [7]:
def parse_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        data = line.strip().split()
        class_label = int(data[0])  # Assuming class label is the first element
        label = 0 if class_label == 0 else 1
        bbox_x_center = float(data[1])
        bbox_y_center = float(data[2])
        bbox_width = float(data[3])
        bbox_height = float(data[4])

        # Convert YOLO format to bounding box coordinates (xmin, ymin, xmax, ymax)
        img_width = 512  # Assuming image size is known
        img_height = 512
        xmin = int((bbox_x_center - bbox_width / 2) * img_width)
        ymin = int((bbox_y_center - bbox_height / 2) * img_height)
        xmax = int((bbox_x_center + bbox_width / 2) * img_width)
        ymax = int((bbox_y_center + bbox_height / 2) * img_height)

        annotations.append((class_label, xmin, ymin, xmax, ymax))

    return annotations

In [8]:
# Load and preprocess image data for training, validation, and testing
train_images_data = [load_and_preprocess_image(image_path) for image_path in train_images]
val_images_data = [load_and_preprocess_image(image_path) for image_path in val_images]

In [9]:
print(f"Total Images (After Preprocessing): {len(images)}")
print(f"Total Annotations (After Preprocessing): {len(annotations)}")

Total Images (After Preprocessing): 1300
Total Annotations (After Preprocessing): 1300


In [10]:
# Parse annotations for training and validation
train_annotations_data = []
val_annotations_data = []

train_annotations_data = [parse_annotation(annotation_path) for annotation_path in train_annotations]
val_annotations_data = [parse_annotation(annotation_path) for annotation_path in val_annotations]

In [11]:
print(f"Total Images (After Preprocessing): {len(train_images_data)}")
print(f"Total Annotations (After Preprocessing): {len(train_annotations_data)}")

Total Images (After Preprocessing): 936
Total Annotations (After Preprocessing): 936


In [12]:
print(f"Total Images (After Preprocessing): {len(train_images_data)}")
print(f"Total Annotations (After Preprocessing): {len(train_annotations_data)}")

[[(1, 161, 76, 453, 381)],
 [(1, 118, 196, 400, 404), (1, 176, 39, 470, 224)],
 [(1, 77, 164, 468, 505), (1, 0, 1, 292, 137)],
 [(1, 143, 157, 509, 416)],
 [(0, 0, 60, 504, 409)],
 [(1, 63, 49, 427, 336)],
 [(1, 178, 40, 371, 456)],
 [(1, 123, 166, 381, 388)],
 [(1, 37, 80, 496, 408)],
 [(0, 5, 18, 424, 468)],
 [(1, 106, 136, 443, 436)],
 [(0, 20, 188, 430, 511), (0, 103, 27, 427, 317)],
 [(1, 41, 34, 497, 484)],
 [(0, 92, 184, 405, 462), (0, 174, 80, 476, 259), (0, 0, 1, 185, 151)],
 [(1, 4, 4, 509, 476)],
 [(0, 47, 35, 436, 471)],
 [(1, 87, 118, 463, 336)],
 [(0, 19, 87, 512, 486)],
 [(1, 81, 132, 412, 362)],
 [(1, 223, 199, 306, 275), (1, 34, 17, 130, 122)],
 [(1, 10, 114, 464, 371)],
 [(0, 27, 3, 468, 512)],
 [(0, 105, 29, 488, 408)],
 [(0, 100, 186, 402, 471), (0, 188, 77, 467, 256)],
 [(1, 54, 181, 233, 392), (1, 326, 162, 443, 323)],
 [(0, 8, 134, 501, 444)],
 [(1, 23, 133, 353, 508)],
 [(0, 49, 90, 432, 417)],
 [(1, 112, 234, 390, 418)],
 [(0, 31, 76, 482, 387)],
 [(1, 167, 232

In [13]:
# Convert lists to NumPy arrays
train_images_data = np.array(train_images_data)
val_images_data = np.array(val_images_data)
# Convert lists to NumPy arrays
train_annotations_data = np.array(train_images_data)
val_annotations_data = np.array(val_images_data)

In [18]:
# Define the CNN model architecture

# Define input shape
input_shape = (img_height, img_width, 3)

# Define the input layer using Input(shape)
input_layer = Input(shape=input_shape)

# Define the rest of the model architecture
model = tf.keras.Sequential([
    input_layer,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Two output neurons for binary classification
])

In [30]:
# Define the model output shape based on the number of classes (2 for binary classification)
output_layer = tf.keras.layers.Dense(3, activation='sigmoid')(input_layer)

# Define the model with the modified output layer
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [32]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [33]:
print("Train Images Data Shape:", train_images_data.shape)
print("Train Annotations Data Shape:", train_annotations_data.shape)

Train Images Data Shape: (936, 512, 512, 3)
Train Annotations Data Shape: (936, 512, 512, 3)


In [34]:
# Train the model
history = model.fit(train_images_data, train_annotations_data, batch_size=batch_size, epochs=10, validation_data=(val_images_data, val_annotations_data))

Epoch 1/10


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 512, 512, 3), output.shape=(None, 512, 512, 3)

In [49]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_annotations, verbose=2)
print("Test accuracy:", test_acc)

ValueError: Unrecognized data type: x=['/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_4685.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_7219.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_5592.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_8863.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_4753.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_1858.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_3682.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_5961.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_7323.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/